In [206]:
import pandas as pd
import numpy as np

In [207]:
# Drop unneeded columns
raw_df = pd.read_csv("../res/responses.csv")
raw_df.drop(["Timestamp", "Name (Optional)"], axis=1, inplace=True);

In [208]:
# Change column names
cols = raw_df.columns
new_cols = []
for col in cols:
    try:
        new_cols.append(col[col.index("[")+1:-1])
    except ValueError:
        new_cols.append(col)
raw_df.columns = new_cols

to_replace = {
    "Would you agree that you frequently shop online?":"Frequency",
    "Where do you shop more often?":"Where",
    "BACK TO SCHOOL - Please select all items that you would typically buy during this season.":"Back To School",
    "SUMMER - Please select all items that you would typically buy during this season.":"Summer",
    "Home Entertainment (TV, Speakers, Online Streaming Subscription, Etc.)":"Home Entertainment"
}
raw_df.rename(columns = to_replace, inplace = True)

raw_df.columns

Index(['Age', 'Gender', 'Marital Status', 'Employment Status',
       'Current Region ', 'Frequency', 'Where', 'Apparel/Clothing',
       'Health & Personal Care', 'Home Entertainment', 'Babies & Kids',
       'Food/Groceries', 'Toys, Games, Collectibles', 'Bags', 'Stationery',
       'Books', 'Sports & Travel', 'Wearable Accessories',
       'Automobile Accessories', 'Cameras/Camera Accessories',
       'Laptops/Computers', 'Tech Accessories', 'Pet Care/Accessories',
       'Power Tools', 'Back To School', 'Summer'],
      dtype='object')

In [209]:
# Create baskets according to season
user_data_cols = ['Age', 'Gender', 'Marital Status', 'Employment Status', 'Current Region ', 'Frequency', 'Where']

# Create baskets for xmas, new year, when needed
holidays_basket = raw_df.drop(["Summer", "Back To School"], axis=1);
bdays_basket = raw_df.drop(["Summer", "Back To School"], axis=1);
periods_basket = raw_df.drop(["Summer", "Back To School"], axis=1);
wneeded_basket = raw_df.drop(["Summer", "Back To School"], axis=1);
seasonal_baskets = [holidays_basket, bdays_basket, periods_basket, wneeded_basket]

# Change values accordingly
keywords = ["Christmas", "Birthdays", "Monthly", "Needed"]
i = 0
for basket in seasonal_baskets:
    cols = basket.columns
    keyword = keywords[i]
    for col in cols:
        if col not in user_data_cols:
            basket[col] = np.where(~(basket[col].str.find(keyword) > -1), 0, 1)
    i += 1 

wneeded_basket.head()

,Age,Gender,Marital Status,Employment Status,Current Region,Frequency,Where,Apparel/Clothing,Health & Personal Care,Home Entertainment,...,Stationery,Books,Sports & Travel,Wearable Accessories,Automobile Accessories,Cameras/Camera Accessories,Laptops/Computers,Tech Accessories,Pet Care/Accessories,Power Tools
0,21,Male,Single,Employed,Region III - Central Luzon,2,Online Stores,1,1,1,...,0,0,0,1,0,0,0,0,0,0
1,21,Male,Single,Student,CAR - Cordillera Administrative Region,1,Online Stores,0,1,1,...,1,1,1,1,1,1,1,1,1,1
2,20,Male,Single,Student,Region III - Central Luzon,2,Online Stores,1,0,0,...,1,1,1,1,1,1,1,0,1,1
3,21,Female,Single,Student,Region IV-A - CALABARZON,4,Online Stores,0,0,1,...,1,1,1,0,0,0,1,1,1,0
4,22,Female,Single,Student,Region IV-A - CALABARZON,5,Online Stores,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [210]:
summer_items = ["Clothing", "Umbrellas", "Sunscreen/Heat-Protectant Skin Products", 
"Swimsuit", "Swim Cap", "Swim Goggles", "Swim Snorkles", "Towels", "Bags", "Water Bottles", 
"Ice Cooler", "Inflatable Pools", "Pool Toys", "Swimming Safety Equipment (Swimming Rings/Life Jacket/Floaties/Etc.)", "Electric Fan"]

bts_items = ["Pen/Marker/Pencil/Scriber", "Paper/Envelopes/Post-it Notes/", "Notebooks", 
"Academic Books", "Art Materials", "Laptop", "Phone/Tablet", "Tech Products/Tech Accessories", 
"School Uniforms/Clothing", "Umbrella", "Raincoat", "Bags/School Bags", "Make up/Beauty Products"]

In [211]:
# Create baskets according to season part 2
summer_basket = raw_df[user_data_cols]
bts_basket = raw_df[user_data_cols]

# Set appropriate columns
summer_basket = summer_basket.reindex(columns = summer_basket.columns.tolist() + summer_items)
bts_basket = bts_basket.reindex(columns = bts_basket.columns.tolist() + bts_items)

# Assign appropriate values in summer basket
for col in summer_basket.columns:
    if col not in user_data_cols:
        summer_basket[col] = np.where(~(raw_df["Summer"].str.find(col) > -1), 0, 1)
# Assign Appropriate Values in Back To School basket
for col in bts_basket.columns:
    if col not in user_data_cols:
        bts_basket[col] = np.where(~(raw_df["Back To School"].str.find(col) > -1), 0, 1)

# Clean column name
to_replace_summer = {
    "Swimming Safety Equipment (Swimming Rings/Life Jacket/Floaties/Etc.)":"Swimming Safety Equipment",
}
summer_basket.rename(columns = to_replace_summer, inplace = True)

summer_basket.head()

,Age,Gender,Marital Status,Employment Status,Current Region,Frequency,Where,Clothing,Umbrellas,Sunscreen/Heat-Protectant Skin Products,...,Swim Goggles,Swim Snorkles,Towels,Bags,Water Bottles,Ice Cooler,Inflatable Pools,Pool Toys,Swimming Safety Equipment,Electric Fan
0,21,Male,Single,Employed,Region III - Central Luzon,2,Online Stores,1,1,0,...,0,0,1,0,1,0,0,0,0,0
1,21,Male,Single,Student,CAR - Cordillera Administrative Region,1,Online Stores,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,20,Male,Single,Student,Region III - Central Luzon,2,Online Stores,1,1,1,...,1,0,0,0,1,0,0,0,0,0
3,21,Female,Single,Student,Region IV-A - CALABARZON,4,Online Stores,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,22,Female,Single,Student,Region IV-A - CALABARZON,5,Online Stores,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [212]:
# List of baskets created:
# holidays_basket
# bdays_basket
# periods_basket
# wneeded_basket
# seasonal_baskets
# summer_basket
# bts_basket